<a href="https://colab.research.google.com/github/Nathbobs/Graduation_Capstone/blob/main/nerf_pineCone_Sample_gen_cam_path_Working_nerfStudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
    <picture>
    <source media="(prefers-color-scheme: dark)" srcset="https://docs.nerf.studio/en/latest/_images/logo-dark.png">
    <source media="(prefers-color-scheme: light)" srcset="https://docs.nerf.studio/en/latest/_images/logo.png">
    <img alt="nerfstudio" src="https://docs.nerf.studio/en/latest/_images/logo.png" width="400">
    </picture>
</p>


# Nerfstudio: A collaboration friendly studio for NeRFs


![GitHub stars](https://img.shields.io/github/stars/nerfstudio-project/nerfstudio?color=gold&style=social)

This colab shows how to train and view NeRFs from Nerfstudio both on pre-made datasets or from your own videos/images.

\\

Credit to [NeX](https://nex-mpi.github.io/) for Google Colab format.

## Frequently Asked Questions

*  **Downloading custom data is stalling (no output):**
    * This is a bug in Colab. The data is processing, but may take a while to complete. You will know processing completed if `data/nerfstudio/custom_data/transforms.json` exists.
*   **Training is not showing progress**:
    * The lack of output is a bug in Colab. You can see the training progress from the viewer.

* **Viewer Quality is bad / Low resolution**:
    * This may be because more GPU is being used on training that rendering the viewer. Try pausing training or decreasing training utilization.

* **Other problems?**
    * Feel free to create an issue on our [GitHub repo](https://github.com/nerfstudio-project/nerfstudio).


In [ ]:
#@title # Install Conda (requires runtime restart) { vertical-output: true, display-mode: "form" }

!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:10
🔁 Restarting kernel...


In [ ]:
#@title # Install Nerfstudio and Dependencies (~10 min) { vertical-output: true, display-mode: "form" }

%cd /content/
!pip install --upgrade pip
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html

# Installing TinyCuda
%cd /content/
!gdown "https://drive.google.com/u/1/uc?id=1q8fuc-Mqiev5GTBTRA5UPgCaQDzuqKqj"
!pip install tinycudann-1.6-cp37-cp37m-linux_x86_64.whl

# Installing COLMAP
%cd /content/
!conda install -c conda-forge colmap

# Install nerfstudio
%cd /content/
!pip install nerfstudio

/content
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.12.1+cu113 (from versions: 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2.1.0+cu121.with.pypi.cudnn, 2.1.0+rocm5.5, 2.1.0+rocm5.6, 2.1.1, 2.1.1+cpu, 2.1.1+cpu.cxx11.abi, 2.1.1+cu118, 2.1.1+cu121, 2.1.1+cu121.with.pypi.cudnn, 2.1.1+roc

In [ ]:
from google.colab import output

# Add this to the end of the cell you want to trigger the sound on
output.eval_js("new Audio('https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg').play()")

In [ ]:
#@markdown <h1> Downloading Data</h1>
#@markdown <h3>Pick the preset scene or upload your own images/video</h3>
import os
from google.colab import files
from IPython.core.display import display, HTML

scene = '📤 upload your images' #@param ['🖼 poster', '🚜 dozer', '🌄 desolation', '📤 upload your images' , '🎥 upload your own video']
scene = ' '.join(scene.split(' ')[1:])

if scene not in ['upload your images', 'upload your own video']:
    %cd /content/
    !ns-download-data --dataset=nerfstudio --capture=$scene
else:
    display(HTML('<h3>Select your custom data</h3>'))
    display(HTML('<p/>You can select multiple images by pressing ctrl, cmd or shift and click.<p>'))
    display(HTML('<p/>Note: This may take time, especially on hires inputs, so we recommend to download dataset after creation.<p>'))
    !mkdir -p /content/data/nerfstudio/pineCone
    if scene == 'upload your images':
        !mkdir -p /content/data/nerfstudio/pineCone/pineCone_images
        %cd /content/data/nerfstudio/pineCone/pineCone_images
        uploaded = files.upload()
        dir = os.getcwd()
    else:
        %cd /content/data/nerfstudio/pineCone/
        uploaded = files.upload()
        dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    del uploaded
    %cd /content/

    if scene == 'upload your images':
        !ns-process-data images --data /content/data/nerfstudio/pineCone/pineCone_images --output-dir /content/data/nerfstudio/custom_data/
    else:
        video_path = preupload_datasets[0]
        !ns-process-data video --data $video_path --output-dir /content/data/nerfstudio/pineCone/

    scene = "custom_data"



from google.colab import output

# Add this to the end of the cell you want to trigger the sound on
output.eval_js("new Audio('https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg').play()")

/content/data/nerfstudio/pineCone/pineCone_images


Saving IMG_7240.png to IMG_7240.png
Saving IMG_7238.png to IMG_7238.png
Saving IMG_7239.png to IMG_7239.png
Saving IMG_7241.png to IMG_7241.png
Saving IMG_7242.png to IMG_7242.png
Saving IMG_7243.png to IMG_7243.png
Saving IMG_7244.png to IMG_7244.png
Saving IMG_7245.png to IMG_7245.png
Saving IMG_7246.png to IMG_7246.png
Saving IMG_7247.png to IMG_7247.png
Saving IMG_7248.png to IMG_7248.png
Saving IMG_7249.png to IMG_7249.png
Saving IMG_7250.png to IMG_7250.png
Saving IMG_7251.png to IMG_7251.png
Saving IMG_7252.png to IMG_7252.png
Saving IMG_7253.png to IMG_7253.png
Saving IMG_7254.png to IMG_7254.png
Saving IMG_7255.png to IMG_7255.png
Saving IMG_7256.png to IMG_7256.png
Saving IMG_7257.png to IMG_7257.png
Saving IMG_7258.png to IMG_7258.png
Saving IMG_7259.png to IMG_7259.png
Saving IMG_7260.png to IMG_7260.png
Saving IMG_7261.png to IMG_7261.png
Saving IMG_7262.png to IMG_7262.png
Saving IMG_7263.png to IMG_7263.png
Saving IMG_7264.png to IMG_7264.png
Saving IMG_7265.png to IMG_7

In [ ]:
# #@title # Set Up Viewer

# %cd /content

# # Install localtunnel
# # We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
# !npm install -g localtunnel

# # Tunnel port 7007, the default for
# !rm url.txt 2> /dev/null
# get_ipython().system_raw('lt --port 7878 >> url.txt 2>&1 &')

In [ ]:
# #@title # Start Viewer

# with open('url.txt') as f:
#   lines = f.readlines()
# websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
# # from nerfstudio.utils.io import load_from_json
# # from pathlib import Path
# # json_filename = "nerfstudio/nerfstudio/viewer/app/package.json"
# # version = load_from_json(Path(json_filename))["version"]
# url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
# print(url)
# print("You may need to click Refresh Page after you start training!")
# from IPython import display
# display.IFrame(src=url, height=800, width="100%")

In [ ]:
#@title # Start Training { vertical-output: true }

%cd /content
!ns-train nerfacto --max-num-iterations 5000 nerfstudio-data --data data/nerfstudio/$scene


from google.colab import output

# Add this to the end of the cell you want to trigger the sound on
output.eval_js("new Audio('https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg').play()")

Streaming output truncated to the last 5000 lines.
1420 (28.40%)       360.207 ms           21 m, 29 s           12.23 K                                
1430 (28.60%)       360.212 ms           21 m, 25 s           12.22 K                                
1440 (28.80%)       356.476 ms           21 m, 9 s            12.38 K                                
1450 (29.00%)       360.644 ms           21 m, 20 s           12.23 K                                
1460 (29.20%)       364.534 ms           21 m, 30 s           12.09 K                                
1470 (29.40%)       359.883 ms           21 m, 10 s           12.28 K                                
1480 (29.60%)       355.636 ms           20 m, 51 s           12.43 K                                
---------------------------------------------------------------------------------------------------- 
Viewer running locally at: http://localhost:7007 (listening on 0.0.0.0)                              
Step (% Done)       Train Iter 

In [ ]:
!apt-get install ffmpeg -y

from google.colab import output

# Add this to the end of the cell you want to trigger the sound on
output.eval_js("new Audio('https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg').play()")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
# %%writefile render_fix.py
# import sys
# import torch
# import numpy
# from nerfstudio.scripts.render import entrypoint as original_entrypoint

# # Patch torch.load to use weights_only=False
# original_load = torch.load
# torch.load = lambda *args, **kwargs: original_load(*args, **{**kwargs, 'weights_only': False})

# # Run the original entrypoint with the arguments passed to this script
# if __name__ == "__main__":
#     sys.exit(original_entrypoint())

Writing render_fix.py


In [ ]:
import json
import numpy as np
import torch
from nerfstudio.cameras.cameras import Cameras
from nerfstudio.cameras.camera_paths import get_spiral_path

# Load camera parameters from transforms.json
json_path = "/content/data/nerfstudio/custom_data/transforms.json"

with open(json_path, "r") as f:
    transforms = json.load(f)


frames = transforms["frames"]
poses = []
focal_lengths = []
image_height = transforms.get("h", 800)
image_width = transforms.get("w", 800)
focal_length = transforms.get("fl_x", 1000)  # Adjust default if needed

for frame in frames:
    pose = torch.tensor(frame["transform_matrix"], dtype=torch.float32)
    poses.append(pose)
    focal_lengths.append(focal_length)

poses = torch.stack(poses)
focal_lengths = torch.tensor(focal_lengths)

# Construct the Cameras object for all cameras (batch)
cameras = Cameras(
    camera_to_worlds=poses,
    fx=focal_lengths,
    fy=focal_lengths,
    cx=image_width / 2,
    cy=image_height / 2,
    width=image_width,
    height=image_height,
    distortion_params=None
)

# ---- KEY FIX: Create a single camera (batch size 1) ----
single_camera = Cameras(
    camera_to_worlds=poses[0].unsqueeze(0),  # Shape: (1, 4, 4)
    fx=focal_lengths[0:1],                   # Shape: (1,)
    fy=focal_lengths[0:1],                   # Shape: (1,)
    cx=image_width / 2,
    cy=image_height / 2,
    width=image_width,
    height=image_height,
    distortion_params=None
)

# Now generate the spiral path
spiral_cameras = get_spiral_path(
    camera=single_camera,
    steps=60,
    radius=1.0,
    rots=2,
    zrate=0.5
)
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return super().default(obj)


def cameras_to_json_fixed(cameras, filename):
    out = {"camera_path": []}
    num_cameras = len(cameras.camera_to_worlds)
    for i in range(num_cameras):
        pose = cameras.camera_to_worlds[i]
        # Convert to numpy array if it's a list
        if isinstance(pose, list):
            pose = np.array(pose)
        elif hasattr(pose, 'cpu'):
            pose = pose.cpu().numpy()
        elif hasattr(pose, 'numpy'):
            pose = pose.numpy()
        else:
            pose = np.array(pose)
        # Ensure pose is 4x4
        if pose.shape == (3, 4):
            bottom = np.array([[0, 0, 0, 1]], dtype=pose.dtype)
            pose = np.vstack([pose, bottom])
        elif pose.shape != (4, 4):
            raise ValueError(f"Unexpected pose shape: {pose.shape}")
        pose_list = pose.tolist()

        # Handle fx, fy, cx, cy as arrays or scalars
        fx = float(cameras.fx[i]) if hasattr(cameras.fx, '__len__') and len(cameras.fx) > 1 else float(cameras.fx)
        fy = float(cameras.fy[i]) if hasattr(cameras.fy, '__len__') and len(cameras.fy) > 1 else float(cameras.fy)
        cx_val = cameras.cx[i] if hasattr(cameras.cx, '__len__') and len(cameras.cx) > 1 else cameras.cx
        cy_val = cameras.cy[i] if hasattr(cameras.cy, '__len__') and len(cameras.cy) > 1 else cameras.cy
        cx = float(cx_val) if not isinstance(cx_val, (float, int)) else cx_val
        cy = float(cy_val) if not isinstance(cy_val, (float, int)) else cy_val

        # Fix width and height conversion
        if hasattr(cameras.width, '__len__') and len(cameras.width) > 1:
            width = int(cameras.width[i])
        elif hasattr(cameras.width, '__len__') and len(cameras.width) == 1:
            width = int(cameras.width[0])
        else:
            width = int(cameras.width)

        if hasattr(cameras.height, '__len__') and len(cameras.height) > 1:
            height = int(cameras.height[i])
        elif hasattr(cameras.height, '__len__') and len(cameras.height) == 1:
            height = int(cameras.height[0])
        else:
            height = int(cameras.height)


        # Compute FOV in degrees
        fov = 2 * np.arctan(width / (2 * fx)) * 180 / np.pi


        out["camera_path"].append({
            "camera_to_world": pose,
            "fx": fx,
            "fy": fy,
            "cx": cx,
            "cy": cy,
            "width": width,
            "height":height,
            "fov": fov
        })
    out["seconds"] = 6.0
    out["render_height"] = height
    out["render_width"] = width
    with open(filename, "w") as f:
        json.dump(out, f, indent=2, cls = NumpyEncoder)
    print("✅ Spiral camera path saved as camera_path.json")

In [ ]:
cameras_to_json_fixed(spiral_cameras, "camera_path.json")

✅ Spiral camera path saved as camera_path.json


In [ ]:
%%writefile render_fix.py
import sys
import torch
import numpy
from nerfstudio.scripts.render import entrypoint as original_entrypoint

# Patch torch.load to use weights_only=False
original_load = torch.load
torch.load = lambda *args, **kwargs: original_load(*args, **{**kwargs, 'weights_only': False})

# Run the original entrypoint with the arguments passed to this script
if __name__ == "__main__":
    sys.exit(original_entrypoint())

Writing render_fix.py


In [ ]:
#@title # Render Video -- My Approach after fixing torch.load to false using a script <br><h5> change "/content/outputs/unnamed/nerfacto/2025-05-23_012600/config.yml" <br>to your own path generated by the previous cell.<h5>
%cd /content

# Create renders directory
!mkdir -p renders

!python render_fix.py camera-path \
  --load-config /content/outputs/unnamed/nerfacto/2025-05-27_221409/config.yml \
  --camera-path-filename /content/camera_path.json \
  --output-path /content/renders/flower_spiral.mp4

# # Download the result
# from google.colab import files
# files.download('renders/flower_spiral.mp4')

/content
/usr/local/lib/python3.11/site-packages/nerfstudio/field_components/activations.py:32: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float32)
/usr/local/lib/python3.11/site-packages/nerfstudio/field_components/activations.py:38: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd

🏃 🏃 Install tcnn for speedups 🏃 🏃
pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch


🏃 🏃 Install tcnn for speedups 🏃 🏃
pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch


🏃 🏃 Install tcnn for speedups 🏃 🏃
pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch


🏃 🏃 Install tcnn for speedups 🏃 🏃
pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch


🏃 🏃

In [ ]:
%cd /content

# Create renders directory
!mkdir -p renders

# Render the spiral (this should work for most scenes)
!ns-render spiral \
    --load-config /content/outputs/unnamed/nerfacto/2025-05-26_033454/config.yml \
    --output-path renders/flower_spiral.mp4 \
    --seconds 5 \
    --output-format video

# # Download the result
# from google.colab import files
# files.download('renders/flower_spiral.mp4')

/content
/usr/local/lib/python3.11/site-packages/nerfstudio/field_components/activations.py:32: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float32)
/usr/local/lib/python3.11/site-packages/nerfstudio/field_components/activations.py:38: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
[04:12:32] Auto image downscale factor of 1                                                 ]8;id=774840;file:///usr/local/lib/python3.11/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=718845;file:///usr/local/lib/python3.11/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\
/usr/local/lib/python3.11/site-packages/nerfstudio/field_components/activations.py:32: FutureWarning: `torch.cuda.a